# Todo
* Sensitivity 250 500 1000
* For the 500, take top 10% heightest weight
* Take that threshold for other topics


In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#import tempfile
#TEMP_FOLDER = tempfile.gettempdir()
#print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('retina')

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.test.utils import datapath

from manage_path import *
from compute_topic import *

### Read the pandas dataframe we got from data preparation

In [3]:
data = load_data('TRACE2014_jinming.pkl')

loading data TRACE2014_jinming.pkl...
Getting data fromC:\Users\raymo\UMD\Research\FINRA_TRACE\Data\Pickle\TRACE2014_jinming.pkl...
Data getting success!


In [ ]:
Dc_v4 = compute_Dc_v4(data)

compute_corpus(Dc_v4,'Dc_v4')
#save_corpus(Dc_v4,'Dc_v4')
compute_id2word(Dc_v4,'Dc_v4')

In [ ]:
Dc_v4.shape

In [ ]:
Dc_v4[Dc_v4.sum(axis=1) > 3].shape

In [ ]:
Dc_v4_sum = Dc_v4.sum(axis=1)

In [ ]:
Dc_v4_sum.describe()

In [ ]:
Dc_v4_sum.sort_values(ascending=False).head(50)

In [ ]:
Dc_v4_sum.sort_values(ascending=False).iloc[100000]

In [ ]:
del Dc_v4_sum

In [2]:
load_corpus('Dc_v4')

loading corpus...
corpus successfully loaded!!
MmCorpus(153916 documents, 12077 features, 6576545 non-zero entries)


In [ ]:
load_corpus('Dc_v3')

In [ ]:
load_corpus('Dc_v2')

In [ ]:
load_corpus('Dc_v1')

## Capitalization

In [6]:
data.head()

,TRD_RPT_DTTM,TRD_EXCTN_DTTM,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,ENTRD_VOL_QT,RPTD_PR,RPT_SIDE_CD,Report_Dealer_Index,...,document_date,ISSUER_ID,COMPLETE_CUSIP,AGENT_ID,CUSIP_NAME,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,SIC_CODE
0,2014-07-16 09:47:25,2014-07-16 09:47:13,T,TWIX3666829,982526AB1,CORP,25000.0,102.991997,B,0,...,2014-07-16,38591,982526AB1,52600,WRIGLEY WM JR CO,1,10,56732,31134,2067
1,2014-12-23 16:11:32,2014-12-23 16:11:17,T,WAMU3885939,92936PAB6,CORP,214.0,95.934578,S,83,...,2014-12-23,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
2,2014-12-05 14:43:44,2014-12-05 14:43:27,T,WAMU3885939,92936PAB6,CORP,100.0,92.500000,S,83,...,2014-12-05,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
3,2014-12-03 10:48:12,2014-12-03 10:48:11,T,WAMU3885939,92936PAB6,CORP,1.0,88.000000,B,0,...,2014-12-03,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
4,2014-12-23 16:11:16,2014-12-23 16:10:56,T,WAMU3885939,92936PAB6,CORP,2000.0,95.250000,S,83,...,2014-12-23,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035


In [7]:
data.columns

Index(['TRD_RPT_DTTM', 'TRD_EXCTN_DTTM', 'TRC_ST', 'BOND_SYM_ID', 'CUSIP_ID',
       'SCRTY_TYPE_CD', 'ENTRD_VOL_QT', 'RPTD_PR', 'RPT_SIDE_CD',
       'Report_Dealer_Index', 'Contra_Party_Index', 'document_date',
       'ISSUER_ID', 'COMPLETE_CUSIP', 'AGENT_ID', 'CUSIP_NAME',
       'INDUSTRY_GROUP', 'INDUSTRY_CODE', 'PARENT_ID', 'NAICS_CODE',
       'SIC_CODE'],
      dtype='object')

In [21]:
data.eval('ENTRD_VOL_QT*RPTD_PR').describe()

count    6.322794e+06
mean     6.092951e+07
std      2.474158e+08
min      0.000000e+00
25%      1.104222e+06
50%      3.386007e+06
75%      2.463350e+07
max      1.154484e+11
dtype: float64

In [14]:
data['ENTRD_VOL_QT'].min()

0.01

In [15]:
data['RPTD_PR'].min()

0.0

In [22]:
data[data['RPTD_PR']<=0]

,TRD_RPT_DTTM,TRD_EXCTN_DTTM,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,ENTRD_VOL_QT,RPTD_PR,RPT_SIDE_CD,Report_Dealer_Index,...,document_date,ISSUER_ID,COMPLETE_CUSIP,AGENT_ID,CUSIP_NAME,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,SIC_CODE
1001650,2014-12-18 12:44:35,2014-12-18 12:44:34,T,DISH3706274,093679AC2,CORP,50000.0,0.0,B,0,...,2014-12-18,37989,093679AC2,51390,BLOCKBUSTER INC,1,15,51390,532282,7841


In [ ]:
data.eval('ENTRD_VOL_QT*RPTD_PR')